In [4]:
from ggplot import *
from tensorflow import keras
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
# import cv2
import pandas as pd
import math
from tensorflow.contrib import distributions
from scipy.misc import logsumexp

In [ ]:
# standard categorical cross entropy
# N data points, C classes
# true - true values. Shape: (N, C)
# pred - predicted values. Shape: (N, C)
# returns - loss (N)
def categorical_cross_entropy(true, pred):
    return np.sum(true * np.log(pred), axis=1)

# Bayesian categorical cross entropy.
# N data points, C classes, T monte carlo simulations
# true - true values. Shape: (N, C)
# pred_var - predicted logit values and variance. Shape: (N, C + 1)
# returns - loss (N,)
def bayesian_categorical_crossentropy(T, num_classes):
    def bayesian_categorical_crossentropy_internal(true, pred_var):
        # shape: (N,)
        std = K.sqrt(pred_var[:, num_classes:])
        # shape: (N,)
        variance = pred_var[:, num_classes]
        variance_depressor = K.exp(variance) - K.ones_like(variance)
        # shape: (N, C)
        pred = pred_var[:, 0:num_classes]
        # shape: (N,)
        undistorted_loss = K.categorical_crossentropy(pred, true, from_logits=True)
        # shape: (T,)
        iterable = K.variable(np.ones(T))
        dist = distributions.Normal(loc=K.zeros_like(std), scale=std)
        monte_carlo_results = K.map_fn(gaussian_categorical_crossentropy(true, pred, dist, undistorted_loss, num_classes), iterable, name='monte_carlo_results')

        variance_loss = K.mean(monte_carlo_results, axis=0) * undistorted_loss

        return variance_loss + undistorted_loss + variance_depressor
  
    return bayesian_categorical_crossentropy_internal

# for a single monte carlo simulation, 
#   calculate categorical_crossentropy of 
#   predicted logit values plus gaussian 
#   noise vs true values.
# true - true values. Shape: (N, C)
# pred - predicted logit values. Shape: (N, C)
# dist - normal distribution to sample from. Shape: (N, C)
# undistorted_loss - the crossentropy loss without variance distortion. Shape: (N,)
# num_classes - the number of classes. C
# returns - total differences for all classes (N,)
def gaussian_categorical_crossentropy(true, pred, dist, undistorted_loss, num_classes):
    def map_fn(i):
        std_samples = K.transpose(dist.sample(num_classes))
        distorted_loss = K.categorical_crossentropy(pred + std_samples, true, from_logits=True)
        diff = undistorted_loss - distorted_loss
        return -K.elu(diff)
    return map_fn

In [15]:
def resnet50(input_shape):
    input_tensor = tf.keras.layers.Input(shape=input_shape)
    base_model = tf.keras.applications.ResNet50(include_top=False, input_tensor=input_tensor)
    # freeze encoder layers to prevent over fitting
    for layer in base_model.layers:
        layer.trainable = False
    output_tensor = tf.keras.layers.Flatten()(base_model.output)
    return tf.keras.Model(inputs=input_tensor, outputs=output_tensor)

In [16]:
resnet50((224, 224, 3))

C:\Users\ipdavies\AppData\Local\Continuum\miniconda3\envs\tensorflow\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
def create_bayesian_model(encoder, input_shape, output_classes):
    encoder_model = tf.keras.applications.ResNet50(input_shape)
    input_tensor = tf.keras.layers.Input(shape=encoder_model.output_shape[1:])
    x = tf.keras.BatchNormalization(name='post_encoder')(input_tensor)
    x = tf.keras.Dropout(0.5)(x)
    x = tf.keras.Dense(500, activation='relu')(x)
    x = tf.keras.BatchNormalization()(x)
    x = tf.keras.Dropout(0.5)(x)
    x = tf.keras.Dense(100, activation='relu')(x)
    x = tf.keras.BatchNormalization()(x)
    x = tf.keras.Dropout(0.5)(x)

    logits = tf.keras.Dense(output_classes)(x)
    variance_pre = tf.keras.Dense(1)(x)
    variance = tf.keras.Activation('softplus', name='variance')(variance_pre)
    logits_variance = tf.keras.concatenate([logits, variance], name='logits_variance')
    softmax_output = tf.keras.Activation('softmax', name='softmax_output')(logits)

    model = tf.keras.Model(inputs=input_tensor, outputs=[logits_variance,softmax_output])

    return model

In [8]:
model.compile(
    optimizer=Adam(lr=1e-3, decay=0.001),
    loss={'logits_variance': bayesian_categorical_crossentropy(100, 10),
          'softmax_output': 'categorical_crossentropy'},
    metrics={'softmax_output': metrics.categorical_accuracy},
    loss_weights={'logits_variance': .2, 'softmax_output': 1.})

NameError: name 'model' is not defined

In [21]:
# flags = tf.app.flags
# FLAGS = flags.FLAGS

# flags.DEFINE_string('dataset', 'cifar10', 'The dataset to train the model on.')
# flags.DEFINE_string('encoder', 'resnet50', 'The encoder model to train from.')
# flags.DEFINE_integer('epochs', 1, 'Number of training examples.')
# flags.DEFINE_integer('monte_carlo_simulations', 100, 'The number of monte carlo simulations to run for the aleatoric categorical crossentroy loss function.')
# flags.DEFINE_integer('batch_size', 32, 'The batch size for the generator')
# flags.DEFINE_boolean('debug', False, 'If this is for debugging the model/training process or not.')
# flags.DEFINE_integer('verbose', 0, 'Whether to use verbose logging when constructing the data object.')
# flags.DEFINE_boolean('stop', True, 'Stop aws instance after finished running.')
# flags.DEFINE_float('min_delta', 0.005, 'Early stopping minimum change value.')
# flags.DEFINE_integer('patience', 20, 'Early stopping epochs patience to wait before stopping.')

min_image_size = encoder_min_input_size(FLAGS.encoder)
((x_train, y_train), (x_test, y_test)) = test_train_batch_data(FLAGS.dataset, FLAGS.encoder, FLAGS.debug, augment_data=True)

min_image_size = list(min_image_size)
min_image_size.append(3)
num_classes = y_train.shape[-1]

NameError: name 'encoder_min_input_size' is not defined

In [ ]:

model = create_bayesian_model(FLAGS.encoder, min_image_size, num_classes)